In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
openFileName = '전자담배 효과_naver_blog_953.txt'
    
f = open(openFileName, 'r', encoding='utf-8')
data = f.read()
f.close()

In [4]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(data)

In [5]:
import gensim

documents2 = [documents1] # gensim.corpora.Dictionary() 는 매개변수로 리스트 타입을 받기 때문에
dictionary = gensim.corpora.Dictionary(documents2)
print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가가': 0, '가감': 1, '가게': 2, '가격': 3, '가경': 4, '가경동': 5, '가계': 6, '가계부채': 7, '가계비': 8, '가공': 9, '가공비': 10, '가관': 11, '가구': 12, '가구점': 13, '가글': 14, '가급': 15, '가기': 16, '가까스로': 17, '가까이': 18, '가끔': 19, '가나': 20, '가난': 21, '가내': 22, '가늠': 23, '가능': 24, '가능성': 25, '가다가': 26, '가닥': 27, '가도': 28, '가독성': 29, '가동': 30, '가득': 31, '가든': 32, '가디안': 33, '가디언': 34, '가뜩이나': 35, '가라': 36, '가락': 37, '가락동': 38, '가락시장역': 39, '가래': 40, '가래침': 41, '가량': 42, '가려웠던': 43, '가려진': 44, '가렵다': 45, '가령': 46, '가로': 47, '가로세로': 48, '가로수': 49, '가로수길': 50, '가루': 51, '가마': 52, '가면': 53, '가면무도회': 54, '가모': 55, '가물': 56, '가믐': 57, '가미': 58, '가바': 59, '가발': 60, '가방': 61, '가변': 62, '가변저항': 63, '가보': 64, '가분': 65, '가비': 66, '가비엠': 67, '가빠오': 68, '가사': 69, '가산': 70, '가성': 71, '가세': 72, '가속': 73, '가수': 74, '가스': 75, '가스렌지': 76, '가스실': 77, '가슴': 78, '가슴속': 79, '가습기': 80, '가시': 81, '가식': 82, '가신': 83, '가안': 84, '가야': 85, '가연': 86, '가연성': 87, '가열': 88, '가오': 89, '가오동': 90, '가요': 91, '가용': 92, '가우리': 93, '가운데': 94, '가운뎃손가락': 95,

In [6]:
corpus_path = '전자담배 효과_naver_blog_953.txt'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [7]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[(105, 3), (191, 1), (451, 2), (970, 1), (1330, 1), (1799, 1), (2301, 2), (2487, 1), (2631, 2), (2634, 2), (2663, 1), (3195, 3), (3728, 1), (3764, 1), (4193, 1), (4677, 1), (5175, 1), (5470, 1), (5987, 1), (7320, 1), (7786, 1), (7902, 1), (8139, 1), (8578, 3), (9124, 1), (10490, 1), (10551, 1), (10696, 1), (10824, 2), (11037, 1), (11387, 2), (11557, 1), (13688, 5), (13817, 1), (14453, 1)]
[]
[]
[(87, 1), (94, 1), (105, 1), (121, 1), (451, 2), (882, 1), (1212, 1), (1325, 1), (1446, 1), (1462, 5), (2210, 1), (2212, 1), (2240, 7), (2279, 4), (2434, 2), (2487, 1), (2605, 3), (2623, 1), (2631, 1), (2634, 2), (2742, 1), (2930, 1), (3195, 1), (3328, 1), (3850, 2), (3953, 1), (4050, 2), (4091, 1), (4280, 1), (5219, 2), (5258, 1), (5933, 1), (6014, 1), (6836, 1), (6883, 1), (7555, 1), (7786, 1), (7990, 1), (8071, 1), (8464, 1), (8516, 1), (8634, 1), (9265, 2), (9524, 1), (9731, 1), (9853, 1), (9944, 2), (9981, 1), (10014, 1), (10103, 1), (10546, 2), (10551, 5), (11146, 1), (11953, 1), (13

[]
[(1005, 1), (1462, 1), (4353, 1), (8648, 1), (9265, 1), (11139, 1)]
[(503, 1), (11611, 1)]
[(105, 1), (3977, 1), (11611, 1), (14341, 1), (14454, 1)]
[(5952, 1), (10551, 1)]
[(6921, 1), (10182, 1), (14454, 1)]
[(688, 1), (2279, 1), (3195, 1), (4088, 1), (5956, 1), (6456, 1), (6519, 1), (7291, 1), (8634, 1), (8831, 1), (10370, 1), (10551, 2), (10980, 1), (11898, 1), (12965, 1), (14453, 1)]
[]
[]
[(451, 1), (1180, 1), (1600, 1), (1915, 1), (1949, 1), (2279, 1), (2301, 1), (2634, 1), (3850, 1), (3876, 1), (4088, 1), (4403, 1), (5015, 1), (5307, 1), (5396, 3), (5522, 1), (6248, 1), (6278, 1), (6340, 2), (6344, 1), (6962, 1), (7139, 1), (7447, 1), (7888, 1), (8027, 5), (8544, 4), (9265, 1), (9481, 1), (9687, 1), (9914, 2), (10258, 1), (10446, 2), (10551, 15), (11260, 1), (11687, 1), (11854, 1), (14092, 1), (14249, 1), (14339, 1), (14455, 1)]
[]
[]
[(460, 1), (949, 1), (1426, 1), (1565, 2), (1588, 1), (2147, 1), (2315, 1), (2434, 2), (2663, 1), (2857, 1), (3704, 2), (4091, 1), (4117, 1), (

[(24, 1), (105, 5), (144, 1), (160, 1), (201, 1), (451, 1), (552, 1), (666, 1), (832, 1), (1335, 1), (1507, 1), (1524, 1), (1550, 1), (1589, 2), (1631, 1), (2029, 1), (2503, 1), (3084, 1), (3760, 1), (3872, 3), (4088, 1), (4237, 2), (4570, 1), (4905, 2), (5360, 1), (5978, 4), (6626, 1), (6631, 1), (7520, 1), (7591, 1), (7786, 3), (8027, 1), (8158, 1), (8270, 1), (8491, 3), (8881, 1), (8984, 1), (9055, 1), (9634, 1), (9655, 1), (9731, 1), (9932, 2), (10107, 1), (10482, 1), (10551, 4), (10597, 1), (10696, 1), (10849, 1), (10872, 1), (12153, 1), (12329, 2), (13537, 2), (13540, 1), (13550, 1), (13817, 2), (14339, 1)]
[]
[]
[(688, 1), (907, 1), (970, 1), (1192, 1), (1588, 2), (1898, 1), (2279, 1), (2434, 1), (3145, 1), (3764, 1), (4088, 1), (5034, 1), (9232, 1), (9748, 1), (9782, 1), (10182, 1), (10458, 1), (10597, 1), (11898, 3), (13817, 1), (14222, 1)]
[(5124, 1), (9889, 1), (10333, 1)]
[(2315, 1), (2434, 1), (8578, 1), (9026, 1)]
[]
[]
[(5332, 1), (6592, 1), (6897, 1), (10551, 1), (14204

[]
[]
[]
[]
[(13619, 1), (14353, 1)]
[(1852, 1), (11107, 1)]
[(5658, 1)]
[]
[(4180, 1), (10816, 1)]
[(13688, 1)]
[]
[(3117, 1)]
[]
[(4128, 1)]
[]
[(687, 1)]
[]
[(1335, 1), (7650, 1)]
[(6823, 1)]
[(5793, 1)]
[]
[]
[(8889, 1)]
[(8409, 1)]
[(3088, 1), (12980, 1)]
[]
[(8710, 1)]
[(8409, 1), (10420, 1)]
[(2434, 1), (13537, 1)]
[(11387, 1)]
[]
[(9731, 1)]
[]
[]
[]
[]
[]
[(10551, 1)]
[(2434, 1)]
[(8889, 1)]
[(13619, 1)]
[(11387, 1)]
[(11838, 1)]
[(2434, 1)]
[]
[(5911, 1), (13569, 1)]
[(10551, 1)]
[]
[(14187, 1)]
[]
[]
[(1335, 1), (5911, 1)]
[(7730, 1), (14012, 1)]
[(6104, 1)]
[]
[(8966, 1)]
[]
[(10696, 1)]
[]
[(8027, 1)]
[]
[]
[(4180, 1)]
[(13619, 1)]
[(8409, 1), (13619, 1)]
[(13550, 1)]
[(8889, 1), (10448, 1)]
[(5939, 1), (8409, 1)]
[]
[(8027, 1)]
[(1335, 1), (8027, 1)]
[]
[(6340, 1)]
[(1115, 1), (8187, 1), (10551, 1)]
[]
[]
[(8409, 1), (13537, 1), (13619, 1)]
[(12098, 1), (12526, 1)]
[]
[(232, 1)]
[]
[(2434, 1), (10696, 1)]
[]
[]
[(14135, 1)]
[(697, 1)]
[(712, 1), (4815, 1), (7269, 1), (112

[(2631, 1), (4570, 1), (10551, 1)]
[(2434, 1), (10696, 1)]
[(7168, 1)]
[(4172, 1)]
[]
[]
[(1333, 1), (10094, 1)]
[(2565, 1)]
[(12174, 1)]
[]
[(105, 1)]
[(1588, 1)]
[(6967, 1)]
[]
[(3195, 1)]
[]
[(7541, 1), (9265, 1)]
[(4570, 1), (10551, 1)]
[(2434, 1), (4668, 1)]
[(11064, 1)]
[]
[]
[]
[]
[(10551, 1)]
[]
[(4254, 1)]
[]
[]
[(8701, 1)]
[(2240, 1), (12067, 1)]
[(1462, 1)]
[]
[(10551, 1)]
[(1335, 1), (12174, 1)]
[(1588, 1), (8027, 1), (9932, 1)]
[]
[(12174, 1)]
[(12174, 1)]
[(53, 1), (1540, 1), (9731, 1), (11437, 1)]
[(2484, 1)]
[]
[]
[]
[]
[]
[]
[]
[(9055, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(6852, 1)]
[]
[]
[]
[]
[(2279, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(2301, 1), (13688, 1)]
[]
[]
[]
[(5238, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[(2934, 1), (10551, 1)]
[]
[(11681, 1)]
[(105, 1), (10551, 1), (11242, 1)]
[(11387, 1)]
[(2528, 1), (10551, 1)]
[]
[(10182, 1), (10551, 1), (12092, 1)]
[(2528, 1)]
[(8421, 1)]
[(94, 1), (144, 1), (10551, 1)]
[(10951, 1)]


...

[(1402, 1)]
[]
[]
[(8907, 1), (10892, 1)]
[]
[(10182, 1)]
[]
[]
[]
[(2484, 1)]
[(1402, 1)]
[]
[(1402, 1)]
[(10192, 1)]
[(1031, 1)]
[(9265, 1), (11423, 1)]
[]
[]
[(4570, 1)]
[(14135, 1)]
[(7706, 1)]
[(3723, 1)]
[]
[(4570, 1), (10546, 1)]
[(2434, 1)]
[]
[(11681, 1)]
[(10654, 1)]
[]
[(4346, 1), (8421, 1), (13263, 1)]
[]
[(5246, 1)]
[(11242, 1)]
[(5246, 1), (6839, 1)]
[(10546, 1)]
[(970, 1)]
[(11815, 1), (13055, 1)]
[(1462, 1), (5246, 1)]
[]
[(1462, 1), (2631, 1)]
[(1462, 1), (2417, 1), (9932, 1), (10546, 2), (14450, 1)]
[(1462, 1), (2213, 1), (2240, 2), (2930, 1), (4050, 1), (7555, 1), (8797, 1), (10546, 1), (13752, 1)]
[(1462, 1), (2631, 1), (5091, 1), (7990, 1), (8509, 2), (9932, 2), (10546, 2)]
[(2625, 1), (4415, 3), (4663, 1), (5896, 1), (6246, 1), (8509, 1), (9932, 1), (10546, 2), (12217, 1), (14453, 1)]
[(12192, 1)]
[(11815, 1)]
[(9932, 1)]
[(459, 1)]
[(10546, 1), (10551, 1), (14453, 1)]
[(8813, 1)]
[(105, 1), (648, 1), (2240, 1), (5171, 1), (6519, 1)]
[(2434, 1), (7738, 1), (8071, 

[(18, 1), (19, 1), (105, 2), (128, 1), (451, 1), (467, 1), (553, 1), (570, 1), (648, 1), (1208, 2), (1335, 3), (1540, 1), (2128, 2), (2279, 5), (2434, 1), (2481, 1), (2559, 1), (2621, 1), (2906, 1), (3605, 1), (3777, 1), (3876, 2), (4055, 1), (4088, 1), (4539, 1), (4570, 1), (4815, 1), (5071, 1), (5109, 1), (5136, 1), (5175, 1), (5252, 1), (5381, 1), (5738, 1), (5895, 1), (5907, 1), (6082, 1), (6268, 1), (6840, 1), (7333, 1), (7786, 2), (7933, 1), (8027, 14), (8270, 1), (8578, 1), (8645, 1), (8718, 1), (8734, 1), (8898, 1), (9062, 1), (9731, 1), (9932, 1), (9986, 1), (10250, 1), (10481, 1), (10490, 1), (10551, 7), (10575, 1), (10607, 1), (10627, 1), (10629, 1), (10686, 2), (10696, 1), (10704, 1), (10980, 2), (11037, 1), (11224, 1), (11281, 1), (11466, 1), (11670, 1), (11899, 1), (11949, 2), (12367, 1), (12583, 1), (12636, 1), (12782, 1), (12991, 1), (13048, 1), (13225, 1), (13234, 1), (13457, 1), (13511, 1), (13537, 3), (13564, 2), (13747, 1), (13815, 1), (14047, 1), (14249, 1)]
[]
[]


[(1985, 1), (2240, 2), (2380, 1), (8027, 1), (10551, 1), (10824, 1)]
[(4262, 1), (11580, 1)]
[(4403, 1)]
[(121, 1), (153, 1), (2634, 1), (10724, 1), (14339, 1)]
[(10182, 1), (10339, 1), (10551, 1)]
[(529, 1)]
[(1325, 1), (1599, 1), (10516, 1), (10708, 1), (10951, 1)]
[(3612, 1), (4680, 1), (10791, 1)]
[(8813, 1)]
[(3802, 1), (8024, 1), (13815, 1)]
[(1005, 1), (7898, 1), (8813, 1), (10147, 1), (10551, 1)]
[(8813, 1), (11084, 1)]
[(1394, 1), (2556, 1), (2852, 1), (5258, 1), (7555, 1)]
[(648, 2)]
[(12070, 1)]
[]
[(1286, 1), (6899, 1), (12611, 1)]
[(619, 1), (5333, 1), (5334, 1), (14205, 1)]
[]
[(8786, 2), (8812, 2), (9968, 1), (14420, 1)]
[]
[]
[(1208, 1), (1426, 1), (1507, 1), (1589, 1), (1678, 1), (2301, 1), (2634, 1), (2930, 1), (3195, 1), (3274, 1), (3872, 1), (3879, 1), (4367, 1), (4905, 1), (5952, 1), (7160, 1), (7786, 1), (8499, 1), (8665, 1), (8696, 2), (8697, 1), (8805, 1), (8831, 1), (8883, 1), (8889, 2), (9938, 2), (10206, 1), (10255, 1), (10481, 1), (10512, 2), (10551, 5), (10

[]
[]
[(18, 1), (105, 2), (121, 1), (395, 1), (451, 3), (580, 1), (599, 1), (648, 2), (970, 1), (1335, 2), (1453, 1), (1820, 1), (1981, 1), (2279, 3), (2284, 1), (2301, 1), (2434, 4), (3337, 1), (3615, 1), (3622, 1), (3872, 2), (4091, 1), (4262, 1), (4982, 1), (5392, 1), (5931, 1), (6069, 3), (7241, 1), (7396, 1), (7512, 1), (7541, 1), (8267, 1), (8270, 1), (9265, 1), (10182, 1), (10192, 1), (10686, 3), (10791, 1), (10824, 1), (10872, 4), (12007, 2), (12174, 3), (12436, 1), (12949, 1), (13401, 1), (13537, 3), (13540, 1), (13550, 1), (13564, 1), (13657, 1)]
[]
[]
[(684, 1), (688, 1), (697, 1), (2279, 1), (4905, 1), (5175, 2), (6257, 1), (6399, 1), (6435, 1), (7755, 1), (7786, 1), (9877, 1), (9914, 1), (10551, 2), (10791, 1), (11027, 1), (13164, 1), (14028, 1)]
[]
[]
[]
[]
[]
[]
[(10849, 1)]
[(4088, 1), (5258, 1)]
[]
[]
[]
[]
[]
[(9054, 1)]
[(2631, 1)]
[]
[(9980, 1), (14028, 1)]
[(6069, 1)]
[(6069, 1), (10551, 1)]
[(6069, 1), (12805, 1)]
[]
[(451, 1)]
[(5663, 1), (11022, 1)]
[]
[]
[]
[(4

[]
[]
[(10551, 1), (11681, 1)]
[(10551, 1), (12174, 1)]
[]
[(3929, 1)]
[(3799, 1)]
[(12174, 1)]
[]
[(7541, 1)]
[]
[(5258, 1), (10551, 1)]
[(2240, 1), (2355, 1), (3689, 1), (10551, 1), (10585, 1), (12174, 1)]
[]
[(2240, 1), (7541, 1)]
[(13540, 1)]
[]
[(2683, 1), (3374, 1), (5636, 1), (8339, 1)]
[(2240, 1), (2355, 1), (10551, 1), (12174, 1)]
[(5015, 1)]
[]
[]
[]
[(10551, 1)]
[(8183, 1)]
[(6404, 1), (10551, 1)]
[(320, 1)]
[(2796, 1), (2934, 1), (10551, 1), (13729, 1)]
[]
[]
[(467, 1), (580, 1), (1013, 1), (1031, 1), (1208, 1), (1462, 1), (1579, 1), (2213, 1), (2240, 1), (2631, 1), (2634, 1), (8634, 2), (9634, 1), (9704, 2), (10551, 3), (10779, 1), (11248, 1), (13480, 1), (13616, 1), (14009, 1)]
[]
[]
[(105, 1), (1476, 1), (2301, 1), (2323, 1), (2424, 1), (2634, 1), (2847, 1), (4194, 1), (6895, 1), (7933, 1), (9054, 4), (9223, 1), (10182, 1), (10322, 1), (11959, 1), (12034, 1), (13537, 1), (13752, 1), (13911, 1), (14339, 3), (14341, 3)]
[]
[]
[(1538, 1), (1608, 1), (1631, 1), (2301, 1), (3

In [8]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=14522, num_topics=10, decay=0.5, chunksize=2000)


In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 14522)
0.9999999


In [11]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

C:\Users\BOGYUNG\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [12]:
pyLDAvis.display(prepared_data)
# pyLDAvis.show(prepared_data)